# Setting Mask for Shetran

This notebook was created to demostrate the process of creating the mask use in SHETRAN based in the catchment extent

---

#### Author: 
                LF Velasquez & I Rohrmueller - Newcastle University

#### Date:
                Nov 2022

#### Version:
                1.0

#### Notes:
                - To get jupyter env version type `!jupyter --version` in a python cell
            
#### Jupyter version:

#### Python version:

---

# Notebook set-up

## 1. Setting Python Modules

In [11]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon
import numpy as np
import numbers
from pathlib import Path

# Custom modules
import utils

# Modules for config.ini
from configparser import ConfigParser
config = ConfigParser()

## 2. Global variables

In [12]:
# Setting the path to the work environment
dir_abs = Path().resolve().parent.parent
dir_abs

# Read config file values
config.read('config.ini')

# Setting CRS
crs_global = config.getint('crs_setting', 'GLB')
crs_local = config.getint('crs_setting', 'COL')

# No data value
ND = config.getint('res_setting', 'NO_DATA')

## 3. Start of Process

### Reading shapefile and create fishnet

In [13]:
# Look and select the right shapefile
p = Path(Path(dir_abs / 'shetran_data/original_data')).glob('**/*.shp')
files = [x for x in p if x.is_file()]

print(f'These are the shapefiles available: \n')
for count, value in enumerate(files):
    print(f'{count} : {value.name} /n')

# Making sure the right value has been entered
while True:
    try:
        selection = input('\n -> Which shapefile you want to use. Enter the number i.e. 0,1,2')
        if isinstance(int(selection), numbers.Real):
            print(f'\n You selected {files[int(selection)].name}')
            break
    except ValueError:
        print("Sorry, your response was not an integer")
    
# Read shp file to geopandas dataframe
catchm = gpd.read_file(files[int(selection)])

# If catchment in WGS need to project before doing anything - EPSG:3116
# this code only transfroms from EPSG 4326 - for other CRS the code will need to be modified
# catchm.crs.to_epsg(20) see https://gis.stackexchange.com/questions/326690/explaining-pyproj-to-epsg-min-confidence-parameter
if catchm.crs.to_epsg(20) == crs_global:
    user_crs = crs_local
    catchm = catchm.to_crs(f'{user_crs}')
    
    # Get the bbox for the polygon
    xmin,ymin,xmax,ymax =  catchm.total_bounds

    # Create the fishnet
    width = config.getint('res_setting', 'WIDTH')
    height = config.getint('res_setting', 'HEIGHT')
    rows = int(np.ceil((ymax-ymin) /  height))
    cols = int(np.ceil((xmax-xmin) / width))
    XleftOrigin = xmin
    XrightOrigin = xmin + width
    YtopOrigin = ymax
    YbottomOrigin = ymax- height
    polygons = []
    for i in range(cols):
        Ytop = YtopOrigin
        Ybottom =YbottomOrigin
        for j in range(rows):
            polygons.append(Polygon([(XleftOrigin, Ytop), (XrightOrigin, Ytop), (XrightOrigin, Ybottom), (XleftOrigin, Ybottom)])) 
            Ytop = Ytop - height
            Ybottom = Ybottom - height
        XleftOrigin = XleftOrigin + width
        XrightOrigin = XrightOrigin + width
else:
    user_crs = catchm.crs.to_epsg(20)
    # Get the bbox for the polygon
    xmin,ymin,xmax,ymax =  catchm.total_bounds

    # Create the fishnet
    width = 2000
    height = 2000
    rows = int(np.ceil((ymax-ymin) /  height))
    cols = int(np.ceil((xmax-xmin) / width))
    XleftOrigin = xmin
    XrightOrigin = xmin + width
    YtopOrigin = ymax
    YbottomOrigin = ymax- height
    polygons = []
    for i in range(cols):
        Ytop = YtopOrigin
        Ybottom =YbottomOrigin
        for j in range(rows):
            polygons.append(Polygon([(XleftOrigin, Ytop), (XrightOrigin, Ytop), (XrightOrigin, Ybottom), (XleftOrigin, Ybottom)])) 
            Ytop = Ytop - height
            Ybottom = Ybottom - height
        XleftOrigin = XleftOrigin + width
        XrightOrigin = XrightOrigin + width
    
print(f'\n --- \n The grid mask has been generated as a list of values\n --- \n')

These are the shapefiles available: 

0 : hybas_sa_lev06_v1c_santiagoCali_cauca.shp /n
1 : hybas_sa_lev10_v1c.shp /n
2 : hybas_sa_lev06_v1c.shp /n
3 : hybas_sa_lev08_v1c.shp /n
4 : hybas_sa_lev06_v1c_cauca.shp /n
5 : catchm_boundary.shp /n
6 : WFD_River_Water_Body_Catchments_Cycle_3.shp /n
Sorry, your response was not an integer

 You selected hybas_sa_lev06_v1c_santiagoCali_cauca.shp

 --- 
 The grid mask has been generated as a list of values
 --- 



### Working with the fishnet to create the SHETRAN ID and the CSV File

In [14]:
# Create grid as geopandas dataframe
grid = gpd.GeoDataFrame({'geometry':polygons})
grid.set_crs(epsg=f'{user_crs}', inplace=True)

# Intersect grid with catchment to add SHETRAN ID
# catchm is in the projected crs from the section above
intersection = grid['geometry'].intersects(catchm.loc[0,'geometry'])

# Add the intersection to the grid geodataframe
grid['intersect'] = intersection

# Assing SHETRAN ID if value is True
grid["SHETRAN_ID"] = np.where(grid['intersect'] == True, 0, ND)

# Save grid file as shp file - this will be an input on other scripts
# add CRS before saving
grid_wgs84 = grid.copy()
# Change CRS to EPSG: 4326
grid_wgs84 = grid.to_crs(crs_global)

grid_wgs84.to_file(Path(dir_abs / 'shetran_data/active_data/final_mask_wgs84.shp'))


# Get the centroid for each grid
grid['centroid'] = grid['geometry'].centroid

# Create lat (Y) and lon (X) columns
grid['lat'] = grid['centroid'].y.astype(int)
grid['lon'] = grid['centroid'].x.astype(int)

# Change geopandas to pandas ready to create csv file
df_grid = pd.DataFrame(grid[['SHETRAN_ID', 'lat', 'lon']].copy())
df_grid

# Pivoting dataframe to replicate SHETRAN format
# Pivoting dataframe using lon as column and lat as row
df_pivot = df_grid.pivot(index='lat', columns='lon', values='SHETRAN_ID')
df_pivot = df_pivot.sort_index(ascending=False)


grid

,geometry,intersect,SHETRAN_ID,centroid,lat,lon
0,"POLYGON ((708018.218 889266.403, 710018.218 88...",False,-9999,POINT (709018.218 888266.403),888266,709018
1,"POLYGON ((708018.218 887266.403, 710018.218 88...",False,-9999,POINT (709018.218 886266.403),886266,709018
2,"POLYGON ((708018.218 885266.403, 710018.218 88...",False,-9999,POINT (709018.218 884266.403),884266,709018
3,"POLYGON ((708018.218 883266.403, 710018.218 88...",False,-9999,POINT (709018.218 882266.403),882266,709018
4,"POLYGON ((708018.218 881266.403, 710018.218 88...",False,-9999,POINT (709018.218 880266.403),880266,709018
...,...,...,...,...,...,...
793,"POLYGON ((782018.218 857266.403, 784018.218 85...",True,0,POINT (783018.218 856266.403),856266,783018
794,"POLYGON ((782018.218 855266.403, 784018.218 85...",False,-9999,POINT (783018.218 854266.403),854266,783018
795,"POLYGON ((782018.218 853266.403, 784018.218 85...",False,-9999,POINT (783018.218 852266.403),852266,783018
796,"POLYGON ((782018.218 851266.403, 784018.218 85...",False,-9999,POINT (783018.218 850266.403),850266,783018


### Saving the final outpt as csv file

In [15]:
# Creating the text file for min elevation
utils.shetran_csv_file(dir_abs, 'final_catchment_mask_SHETRAN', df_pivot, 'd')
print('SHETRAN maks file created')

SHETRAN maks file created
